<a href="https://colab.research.google.com/github/chandradutta/DL_Assignment_2/blob/main/DL_Assignment2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision.models as models
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,Subset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torch.nn.functional as func
# import wandb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
import torchvision

In [ ]:
#installing wandb
!pip install wandb
import wandb
from wandb.keras import WandbCallback
import socket
socket.setdefaulttimeout(30)
wandb.login()
wandb.init(project='DL_Assignment_2')

In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2024-04-11 19:36:49--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c03::cf, 2607:f8b0:4023:c06::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  86.8MB/s    in 40s     

2024-04-11 19:37:30 (90.3 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [ ]:
!rm nature_12K.zip

In [ ]:
dataset = torchvision.datasets.ImageFolder(root='inaturalist_12K/train', transform=transforms)
from torchvision import transforms

transform = transforms.Compose([transforms.Resize((200,200)),transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229, 0.224, 0.225])])

train_dataset = ImageFolder(root='inaturalist_12K/train', transform=transform)
# test_dataset = ImageFolder(root='/kaggle/input/inaturalist-dataset/inaturalist_12K/val', transform=transform)


# Define the size of the validation set (20%)
validation_size = 0.2

# Split the dataset into training and validation sets with shuffling
train_data, val_data = train_test_split(list(range(len(train_dataset))), test_size=validation_size, shuffle=True, random_state=42)
#Data Loader

train_subset = Subset(train_dataset, train_data)
val_subset  = Subset(train_dataset, val_data)


In [ ]:
from tqdm import tqdm

In [ ]:
def DataLoaders(aug,batch_size):
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=True)
    return train_loader,val_loader

In [ ]:
def RESNET50_1(k,NUM_OF_CLASSES): #this function returns the model by freezing first k layers only
    model = models.resnet50(pretrained=True)

    params = list(model.parameters())
    for param in params[:k]:
        param.requires_grad = False #freezing

    num_ftrs = model.fc.in_features

    model.fc = torch.nn.Linear(num_ftrs, NUM_OF_CLASSES)

    return model

In [ ]:
LossFun = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # integrating with gpu

def train_model(epochs,activationFun,aug,batch_size,eta):
    train_loader,val_loader = DataLoaders(aug,batch_size)
    Network = RESNET50_1(5,10).to(device)
    Algo = torch.optim.Adam(Network.parameters(), lr=eta)
    for epoch in range(epochs):
        for i,(images,labels) in enumerate(tqdm(train_loader)):

            images = images.to(device)
            labels = labels.to(device)
            # Forward pass
            outputs = Network(images)
            loss = LossFun(outputs, labels)

            # Backward and optimize
            Network.zero_grad()
            loss.backward()
            Algo.step()
        train_acc,train_loss = calculate_accuracy(Network,train_loader,activationFun)
        Validation_acc,val_loss=calculate_accuracy(Network, val_loader,activationFun)
        # wandb.log({"Validation_acc" : Validation_acc})
        # wandb.log({"val_loss" : val_loss})
        # wandb.log({"train_acc" : train_acc})
        # wandb.log({"train_loss" : train_loss})
        print(train_acc,train_loss)
        print(Validation_acc,val_loss)


#accuracy
def calculate_accuracy(model, data_loader,activationFun):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    num_loss=0
    total_length = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images=images.to(device=device)
            labels=labels.to(device=device)
            outputs = model(images,activationFun)
            loss=LossFun(outputs,labels)
            num_loss+=loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_length += predicted.size(0)
    accuracy = correct / total
    loss=num_loss/total_length
    model.train()
    return accuracy,loss

In [ ]:
batch_size = 32
epochs = 10
eta = 1e-4
activationFun='relu'
aug = 'no'
# NUM_OF_CLASSES = 10
# strategy = 2
train_model(epochs,activationFun,aug,batch_size,eta)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 67.5MB/s]
  1%|          | 2/250 [01:01<2:06:28, 30.60s/it]


KeyboardInterrupt: 

In [ ]:
def main_fun():
    wandb.init(project ='DL_Assignment_2B')
    params = wandb.config
    with wandb.init(project = 'DL_Assignment_2B', name='strategy'+str(params.strategy)+'batchSize'+str(params.batchSize)+'num_epochs'+str(params.num_epochs)+'learning_rate'+str(params.learning_rate)) as run:
        train_model(params.num_epochs,params.aug,params.batchSize,params.learning_rate)
sweep_params = {
    'method' : 'bayes',
    'name'   : 'Accuracy',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'val_accuracy',
    },
    'parameters' : {
        'batchSize':{'values':[32,64]},
        'num_epochs':{'values':[10,5]},
        'learning_rate':{'values' : [1e-3,1e-4]},
        'aug' :{'values':['no','yes']},
    }
}
sweepId = wandb.sweep(sweep_params,project = 'DL_Assignment_2B')
wandb.agent(sweepId,function =main_fun,count = 2)
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: pq25pp5r
Sweep URL: https://wandb.ai/dlassignment/DL_assignment_2_B/sweeps/pq25pp5r


wandb: Agent Starting Run: na2eekxx with config:
wandb: 	aug: no
wandb: 	batchSize: 32
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 10
wandb: 	strategy: 2


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

100%|██████████| 125/125 [00:45<00:00,  2.77it/s]


train_accuracy:79.7250,train_loss:4796.9753
validation_accuracy:75.2876,validation_loss:1517.3449


train_accuracy,▁▄▆▇█
train_loss,█▅▃▂▁
val_accuracy,▁▄▇▇█
val_loss,█▅▂▂▁
train_accuracy,79.725
train_loss,4796.97534
val_accuracy,75.28764
val_loss,1517.34486


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7c7f283a9600>> (for post_run_cell), with arguments args (<ExecutionResult object at 7c7df0594d90, execution_count=14 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7c7df0596da0, raw_cell="def main_fun():
    wandb.init(project ='DL_assign.." store_history=True silent=False shell_futures=True cell_id=9426ab7b-71f6-4fe9-8bb5-aadaf8ab4872> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
import pandas as pd
data = {'LayerName': ['Image', 'conv1', 'pool1','conv2','pool2','conv3','pool3','conv4','pool4','conv5','pool5','DenseLayer','Output'],
        'Complete Tensor Size': ['256*256*3','((257-1k)^2)*m','((258-2k)^2)*m','((259-3k)^2)*m','((260-4k)^2)*m','((261-5k)^2)*m','((262-6k)^2)*m','((263-7k)^2)*m','((264-8k)^2)*m','((265-9k)^2)*m','((266-10k)^2)*m','((266-10k)^2)*m*n','n*10'],
        'Multiplications': ['0','3m(k^2)*(257-k)^2','0','(k^2*m^2)*(259-3k)^2','0','(k^2*m^2)*(261-5k)^2','0','(k^2*m^2)*(263-7k)^2','0','(k^2*m^2)*(265-9k)^2','0','(266-10k)^2*m*n','n*10'],
        'Additions': ['0','3m(k^2-1)*(257-k)^2','0','m^2*(k^2-1)*(259-3k)^2','0','(m^2)*(k^2-1)*(261-5k)^2','0','(m^2)*(k^2-1)*(263-7k)^2','0','(m^2)*(k^2-1)*(265-9k)^2','0','n*((266-10k)^2*m-1)','10*(n-1)'],
       }

df = pd.DataFrame(data)
df.head(13)

import matplotlib.pyplot as plt

# Plot the DataFrame as a table
plt.figure(figsize=(15,5))
ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table = plt.table(cellText=df.values, colLabels=df.columns, loc='center')
table.scale(1, 1.5)
table.auto_set_font_size(False)
table.set_fontsize(10)
plt.title('Computations')
plt.savefig('table4.png', bbox_inches='tight', pad_inches=0.05)
wandb.log({"Computations4": wandb.Image('table4.png')})
plt.show()
